In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os
import glob
import math
from sklearn.utils import class_weight
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
# from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers.core import Flatten, Dense, Dropout
# from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
# from keras.optimizers import SGD, Adam, Adamax,rmsprop
# from keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf
from PIL import Image
#from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
#from sklearn.utils import class_weight
#from sklearn.preprocessing import OneHotEncoder
# from catboost import CatBoostClassifier
#import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
# from keras.models import Sequential
# from keras.layers.core import Flatten, Dense, Dropout
# from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
# from keras.optimizers import SGD, Adam, Adamax,rmsprop
# from keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.python.keras.utils.data_utils import Sequence
# from imblearn.over_sampling import SMOTE
# from imblearn.keras import balanced_batch_generator
# from imblearn.under_sampling import  RandomUnderSampler
from lightgbm import LGBMClassifier
# from imblearn.ensemble import BalancedRandomForestClassifier
# from imblearn.over_sampling import  RandomOverSampler
# import pickle

In [ ]:
!pip install lightgbm

In [ ]:
dataset=pd.read_csv('../input/diabetic-retinopathy-resized/trainLabels_cropped.csv')
print(dataset.head(10))

In [ ]:
dataset.drop(columns={'Unnamed: 0','Unnamed: 0.1'},axis=1,inplace=True)
print(dataset.head(10))

In [ ]:
print(dataset.isnull().sum()/len(dataset))

In [ ]:
print(dataset.dtypes)

In [ ]:
print(len(dataset))

In [ ]:
dataset['level'].value_counts().plot(kind='bar')
plt.xlabel('Diabetic Risk levels')
plt.ylabel('Total Image count')

In [ ]:
train_dir='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped'
train_img=['../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/{}'.format(i) for i in os.listdir(train_dir)]

In [ ]:
train_img[:5]

In [ ]:
def plot_image():
  _, axs = plt.subplots(5, 5, figsize=(20, 20))
  axs = axs.flatten()  
  for img,ax in zip(train_img,axs):  
      image=cv2.imread(img) 
      # show the plotting graph of an image  
      image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB) 
      ax.imshow(image)     
  plt.show()

plot_image()

In [ ]:
# Visualizing the Intensity Histogram of Images
def draw_intensity_plot():
  _, axs = plt.subplots(10, 10, figsize=(25, 25))
  axs = axs.flatten()  
  for img,ax in zip(train_img,axs):
    img = cv2.imread(img)
    image=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax.hist(image.ravel(),256,[0,256])
  plt.show()

draw_intensity_plot()

In [ ]:
# Histogram Equalization
def histogram_equalize(img): 
  for img in train_img[4:10]:
    img=cv2.imread(img)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv=cv2.resize(img_yuv,(300,300))
    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR) 
    #res = np.hstack((image, equ))
    plt.hist(img_output.ravel(),256,[0,256])
    plt.show()
histogram_equalize(train_img)

In [ ]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()
vgg16_feature_list = []

show = 0
for img_name in dataset.image+str('.jpeg'):
    if show%7000 == 0:print(show//350) #7000 20%
    show += 1
    img_path=os.path.join(train_dir,img_name)  
    img = image.load_img(img_path, target_size=(200, 200))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feature_list.append(vgg16_feature_np.flatten())

train = np.array(vgg16_feature_list)

In [ ]:
train

In [ ]:
train_y=dataset.loc[:,'level']
train_y=train_y.astype('category')
print(train_y)

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(train,train_y,test_size=0.30)

In [ ]:
import gc
del train

In [ ]:
gc.collect()#freee memory

In [ ]:
#test_x,val_x,test_y,val_y=train_test_split(train_x,train_y,test_size=0.30)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
#print('Class weights:',class_weights)
class_weights={0:0.2726,1:2.8962,2:1.3141,3:8.004,4:9.9092}

In [ ]:
params = {
	# ...SS
    'objective': 'multiclass',
    'num_class':5,
    'metric': 'multi_logloss',
    'class_weight':class_weights
}

# classifier.fit(train_x,train_y)
classifier= LGBMClassifier(boosting_type='gbdt',objective='multiclass',num_class=5,metric='multi_logloss',class_weight=class_weights)
classifier.fit(train_x,train_y)

In [ ]:
y_pred_train=classifier.predict(train_x)

# # Predicting the Test set results
y_pred_test=classifier.predict(test_x)

In [ ]:
print("Training Accuracy is:",accuracy_score(train_y,y_pred_train))

print("Testing Accuracy is:",accuracy_score(test_y,y_pred_test))

In [ ]:
cm=confusion_matrix(y_target=train_y,y_predicted=y_pred_train)

In [ ]:
fig,ax=plot_confusion_matrix(conf_mat=cm,cmap='Dark2')

plt.show()

In [ ]:
cm=confusion_matrix(y_target=test_y,y_predicted=y_pred_test)
fig,ax=plot_confusion_matrix(conf_mat=cm,cmap='Dark2')
plt.show()

In [ ]:
# Training Classification Report

print(classification_report(train_y,y_pred_train))

In [ ]:
# Testing Classification Report
print("Testing Classification Report")
print(classification_report(test_y,y_pred_test))